Example: Running the Stokes code to explore topographic decay on floating viscous sheets
for an ensemble of wavelength perturbations

The code requires FEniCS---see the README for details.

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, '../source')

Install ipyparallel if needed (then restart kernel):

In [2]:
# !pip install --user ipyparallel

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import ipyparallel as ipp
import os 
from params import t_e,H
from post_process import get_decay_rate
from theory import t_p, t_m

In [4]:
num = 20 # number of engines
mycluster = ipp.Cluster(n = num,timeout=180)
rc = mycluster.start_and_connect_sync()
view = rc.load_balanced_view()
dview = rc[:]
dview.block = True
dview.execute('import sys')
dview.execute('import numpy as np')
dview.execute('sys.path.insert(0, "../source")')
dview.execute('from main import solve')
dview.execute('from params import H')
dview.execute('from mesh_routine import deform_mesh')
dview.execute('from dolfinx.mesh import create_rectangle')
dview.execute('from ufl import cos')
dview.execute('from mpi4py import MPI')

Starting 20 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


  0%|          | 0/20 [00:00<?, ?engine/s]

<AsyncResult(execute): finished>

Define time stepping parameters:

In [5]:
# Time-stepping parameters
t_f = 30*t_e                    # Final time (in terms of intrinsic timescale)
nt = 100*int(t_f/t_e)           # Number of time steps
t = np.linspace(0,t_f, nt)      # timesteps array

In [6]:
# lamdas = np.sort(np.append(np.logspace(-1.2,2.75,50),np.logspace(0.25,1,50)))
lamdas = np.logspace(-1.2,2.75,50) # need less points for base perturbation
mydict = dict(lamdas = lamdas,t=t)
dview.push(mydict);

Define wrapper function for solver in example notebook:

**Note:** For base perturbations, switch the perturb_h and perturb_s in the function below

In [7]:
def wrapper(i):

    # # sinusoidal anomaly for perturbing domain
    lamda = lamdas[i]*H                 # wavelength
    k = (2*np.pi/lamda)                 # wavenumber
    perturb_h = lambda x: 5*cos(k*x)     # surface perturbation
    perturb_s = lambda x: 1e-20*cos(k*x) # base perturbation
    perturb = [perturb_h,perturb_s]

    # Mesh parameters
    L = 20*lamda                        # length of domain
    Nx = 500                            # Number of elements in x direction
    Nz = 10                             # Number of elements in z direction

    # create domain
    domain = create_rectangle(MPI.COMM_WORLD,[[-L/2.0,0.0],[L/2.0,H]], [Nx, Nz])

    # deform the upper surface of the domain accoriding to the perturbation
    domain = deform_mesh(domain,perturb)
    
    # solve the problem
    h,s,x = solve(domain,t)
    
    h_max = np.max(np.abs(h-H)/H,axis=0)
    s_max = np.max(np.abs(s)/H,axis=0)
    
    return([h_max,s_max])

Run the ensemble and return the surfaces

In [ ]:
parameters = list(range(lamdas.size))
async_results = []
for i in parameters:
    async_result = view.apply_async(wrapper, i)
    async_results.append(async_result)

rc.wait_interactive(async_results)

results = [ar.get() for ar in async_results]

unknown:   0%|          | 0/50 [00:00<?, ?tasks/s]

Save results:

In [ ]:
fname = 'results_base.npy'
res = {'results':results,'lamdas':lamdas,'t':t}
np.save(fname,res)

Currently loading and plotting results in the plotting.ipynb notebook